In [3]:
import os
import sys
import time
import json
import pprint
import logging

start = time.time()
pp = pprint.PrettyPrinter(indent = 2)
logger = logging.getLogger("reddit")

with open("../creds.json", "r") as g:
    conf = json.load(g)
    for k in conf.keys():
        print("key: {}".format(k))

key: twitter-api-key
key: twitter-secret-key
key: twitter-bearer
key: twitter-access-token
key: twitter-secret-access
key: aws-access-key
key: aws-secret-access-key


In [9]:
import boto3
import pyspark

secretmanager = boto3.client("secretsmanager")

subreddit = "technology"
spark_host = "spark-master" 
extra_jar_list = "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.2,org.apache.hadoop:hadoop-common:3.3.4,org.apache.hadoop:hadoop-aws:3.3.4,org.apache.hadoop:hadoop-client:3.3.4,io.delta:delta-core_2.12:2.2.0,org.postgresql:postgresql:42.5.0"
try:

    aws_client = secretmanager.get_secret_value(SecretId = "AWS_ACCESS_KEY_ID")
    aws_secret = secretmanager.get_secret_value(SecretId = "AWS_SECRET_ACCESS_KEY")

except:
    aws_client = ""
    aws_secret = ""

spark = pyspark.sql.SparkSession.builder \
                .master("spark://{}:7077".format(spark_host)) \
                .config("spark.scheduler.mode", "FAIR") \
                .config("spark.scheduler.allocation.file", "file:///opt/workspace/redditStreaming/fairscheduler.xml") \
                .config("spark.executor.memory", "2048m") \
                .config("spark.executor.cores", "2") \
                .config("spark.streaming.concurrentJobs", "8") \
                .config("spark.local.dir", "/opt/workspace/tmp/driver/{}/".format(subreddit)) \
                .config("spark.worker.dir", "/opt/workspace/tmp/executor/{}/".format(subreddit)) \
                .config("spark.eventLog.enabled", "true") \
                .config("spark.eventLog.dir", "file:///opt/workspace/events/{}/".format(subreddit)) \
                .config("spark.sql.debug.maxToStringFields", 1000) \
                .config("spark.jars.packages", extra_jar_list) \
                .config("spark.hadoop.fs.s3a.access.key", aws_client) \
                .config("spark.hadoop.fs.s3a.secret.key", aws_secret) \
                .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
                .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider') \
                .config('spark.hadoop.fs.s3a.buffer.dir', '/opt/workspace/tmp/blocks') \
                .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
                .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
                .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") \
                .enableHiveSupport() \
                .getOrCreate()

logger.info("created spark session.")

:: loading settings :: url = jar:file:/Users/stevenhurwitt/Documents/twitter.ai/twitter.ai/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/stevenhurwitt/.ivy2/cache
The jars for the packages stored in: /Users/stevenhurwitt/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.hadoop#hadoop-common added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.hadoop#hadoop-client added as a dependency
io.delta#delta-core_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-97fef6f0-8a6b-48f1-a975-86ddac8d6f86;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.2 in local-m2-cache
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.2 in local-m2-cache
	found org.apache.kafka#kafka-clients;2.8.1 in local-m2-cache
	found org.lz4#lz4-java;1.8.0 in local-m2-cache
	found org.xerial.snappy#snappy-java;1.1.8.4 in local-m2-cache
	found org.slf4j#slf4j-api;1.7.32 in local-m2-cache
	found org.apache.hadoop#hadoop-client-runti

RuntimeError: Java gateway process exited before sending its port number

In [6]:
bucket = "reddit-streaming-stevenhurwitt-2"
subreddit = "technology"
file_path = f"s3a://{bucket}/{subreddit}_clean"
print("reading from {}.".format(file_path))

reading from s3a://reddit-streaming-stevenhurwitt-2/technology_clean.


In [7]:
df = spark.read.format("delta").load(file_path)
df.head()

NameError: name 'spark' is not defined